In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.api.types import is_string_dtype
from typing import List, Set, Dict, Tuple, Optional, Union
from nltk.tokenize import word_tokenize 
import ftfy
import textacy
from textacy import preprocess
import torchtext
from torchtext.data import Field
from utils import io
from torchtext.data import Field, Dataset, Example
import csv


In [6]:
              
def prepare_csv(csv_file:str, path='MBTI', sep='\t', text_col:int=1, label_cols:int=[0], header=None):
    """TODO: optimize for speed and memory use
        Process a csv (usually a Pandas-generated one) in such a way that torchtext can handle it
    """
    df = pd.read_csv(Path(path)/csv_file, sep=sep, header=header)
    df[1] = df[text_col].str.replace("\n", " ")
    df[1] =df[text_col].apply(preprocess)
    df[0] = df.apply(lambda row: ''.join([str(lbl) for lbl in row[label_cols]]), axis=1)
    df = df[[0, 1]]
    df.to_csv(Path(path)/'tmp.tsv', sep='\t', header=False, index=False, index_label=None)


In [24]:
def to_tsv(outfpath, labels:list, texts:list):
    with open(outfpath, 'w', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        for label, text in zip(labels, texts):
            writer.writerow([label, text])

In [19]:
df = pd.read_csv('new.tsv', header=None, sep='\t')
len(df)

46578

In [20]:
df[0]=df[0].astype('str')
df[0]=df[0].apply(lambda x: x if len(x) == 4 else ''.join(['0' for i in range(4-len(x))])+str(x))
df.tail()

,0,1
46573,0111,an imagination I like a girl that does n't lik...
46574,0111,' There 's no conspiracy . People do n't tell ...
46575,0111,. The only thing that has worked for me thus f...
46576,0111,ca n't be bothered to put in the effort . List...
46577,0111,"the deep , meaningful , friendship Yesjustan o..."


In [21]:
df[1]=df[1].apply(ftfy.fix_text)
df[1]=df[1].apply(lambda text: textacy.preprocess_text(text,lowercase=False, no_currency_symbols=True,no_urls=True,no_emails=True,no_phone_numbers=True,no_accents=True,no_contractions=False))
df[1] = df[1].apply(lambda text: text.replace('\"', ''))
df[1] = df[1].apply(lambda text: text.replace('"', ''))
df[1] = df[1].apply(lambda text: ' '.join(text.split()).strip())
df[1]=df[1].apply(ftfy.fix_text)



In [22]:


df

,0,1
0,1000,@ USER @ USER @ USER would be great or @ USER ...
1,1000,sarah manning calls for @ USER : the pomplamoo...
2,1000,i 'm a victim of sexual assault & amp ; the la...
3,1000,! ! ! ! ! hill staffers walked out today in pr...
4,1000,with one beautiful hashtag ( via @ USER ) @ UR...
5,1000,wait to partner even more ! xx @ USER @ USER t...
6,1000,xx @ USER @ USER thank you fam ! xx @ USER @ U...
7,1000,rape of texas teen jada @ URL / p0hfdc2wmp @ H...
8,1000,pneumonia in the hospital . he carried me on h...
9,1000,more soon ! merry christmas ! ! ! ! @ USER wil...


In [26]:
df=df.sample(frac=1.0)
lbls = df[0].tolist()
txts= df[1].tolist()
to_tsv('raw.tsv', lbls[:5000],txts[:5000] )

In [34]:
df.to_csv('tnew.tsv', index=False, header=False, index_label=None, sep='\t', quoting=csv.QUOTE_NONNUMERIC)
df=pd.read_csv('tnew.tsv', sep='\t', header=None)
df.tail()

,0,1
46573,111,an imagination I like a girl that does n't lik...
46574,111,' There 's no conspiracy . People do n't tell ...
46575,111,. The only thing that has worked for me thus f...
46576,111,ca n't be bothered to put in the effort . List...
46577,111,"the deep , meaningful , friendship Yesjustan o..."


In [ ]:
df['text']=df['text'].apply(preprocess)
df

In [33]:
ds=DataFrameDataset(df,  { 'label' : LABEL, 'text' : TEXT })
ds

In [45]:
! ls

LICENSE      common    model_checkpoints  setup.py
README.md    datasets  models		  tasks
__init__.py  dev.tsv   prepare.ipynb	  tensorboard_logs
cache	     docs      requirements.txt   utils


In [ ]:
df=pd.read_csv('dev.tsv', sep='\t', header=None)

df

In [39]:
df['label']=df['label'].apply(lambda x: str(x))


In [ ]:
df['label']=df['MBTI']
df=df[['label', 'text']]
df

In [11]:
txt=s.pop()
txt

'1000\t"@ USER breaking a nail make the hardest nigga feel like a str 8 bitch ever seen them babies staring you dead in your face ? shits a real nursery . they ca n\'t stay tho . @ USER guess its the same way with a cut , niggas superman invincible vibe dies real quick . lmao @ USER word ca n\'t count how many times i \'ve deleted a tweet since this is a "" business account "" . gotta rethink your brand / message always . @ HASHTAG @ HASHTAG @ HASHTAG @ HASHTAG @ HASHTAG etc , story of my life . creating a track around an artist \'s vocal is the most dope experience . dunno why more artist do n\'t consider the idea ? @ HASHTAG @ HASHTAG "" every day i breathe i represent for the have-nots . "" that yeezus album is super abstract & amp ; offset . it \'s dope though . "" trust in god and your golden , trust in rap and your empty "" i \'ve gotta see that new "" man of steel "" movie , heard it was the best movie this year . just noticed i share a birthday with some serious industry power 

In [12]:
preprocess(txt)

'1000 "@ user breaking a nail make the hardest nigga feel like a str 8 bitch ever seen them babies staring you dead in your face ? shits a real nursery . they ca n\'t stay tho . @ user guess its the same way with a cut , niggas superman invincible vibe dies real quick . lmao @ user word ca n\'t count how many times i \'ve deleted a tweet since this is a "" business account "" . gotta rethink your brand / message always . @ hashtag @ hashtag @ hashtag @ hashtag @ hashtag etc , story of my life . creating a track around an artist \'s vocal is the most dope experience . dunno why more artist do n\'t consider the idea ? @ hashtag @ hashtag "" every day i breathe i represent for the have-nots . "" that yeezus album is super abstract & amp ; offset . it \'s dope though . "" trust in god and your golden , trust in rap and your empty "" i \'ve gotta see that new "" man of steel "" movie , heard it was the best movie this year . just noticed i share a birthday with some serious industry power h

In [6]:
df = pd.read_csv(Path('MBTI') / 'tmp.tsv', sep='\t', header=None)

In [8]:
ds=torchtext.data.TabularDataset('tmp.tsv', format='tsv', fields=[('label',LABEL), ('text', TEXT)], skip_header=True)
ds

In [47]:
df.to_csv('new.tsv', index=False, header=False, index_label=False, sep='\t')

In [30]:
other = pd.read_csv('IMDB/dev.tsv', sep='\t', header=None)
other

,0,1
0,1,Best Animated Film in Creation The combination...
1,100,A masterful remake I actually did not know the...
2,10000000,"Feels like a bad fan fiction, this is not the ..."
3,1000,Here's A Nickel. Now Shut Up! The first time I...
4,10,Maybe We Should Learn to Live with Ants! Decid...
5,100,"Oh, the childhood IMDb Top 250: 21710 years af..."
6,10,Jackson and Blomkamp invent a rich and engagin...
7,100,"No Mister Bond, I expect you to die! I will ad..."
8,100,"Exceeds Expectations, But Does Not Fully Live ..."
9,100000,"Expect little, get even less. Going into this ..."


In [46]:
df

,label,text
0,1000,@ USER @ USER @ USER would be great or @ USER ...
1,1000,sarah manning calls for @ USER : the pomplamoo...
2,1000,i 'm a victim of sexual assault & amp ; the la...
3,1000,! ! ! ! ! hill staffers walked out today in pr...
4,1000,with one beautiful hashtag ( via @ USER ) @ UR...
5,1000,wait to partner even more ! xx @ USER @ USER t...
6,1000,xx @ USER @ USER thank you fam ! xx @ USER @ U...
7,1000,rape of texas teen jada @ URL / p0hfdc2wmp @ H...
8,1000,pneumonia in the hospital . he carried me on h...
9,1000,more soon ! merry christmas ! ! ! ! @ USER wil...


In [8]:
df['label']=df[0].apply(lambda x: str(x))


In [9]:
df['MBTI']=df['label'].apply(lambda x: x if len(x) == 4 else ''.join(['0' for i in range(4-len(x))])+str(x))

In [10]:
df[0]=df['MBTI']
df=df[[0,1]]


In [17]:
t=pd.read_csv('tmp.tsv', header=None, nrows=1000, sep='\t')
t.head(20)

,0,1
0,1000,@ USER @ USER @ USER would be great or @ USER ...
1,1000,sarah manning calls for @ USER : the pomplamoo...
2,1000,i 'm a victim of sexual assault & amp ; the la...
3,1000,! ! ! ! ! hill staffers walked out today in pr...
4,1000,with one beautiful hashtag ( via @ USER ) @ UR...
5,1000,wait to partner even more ! xx @ USER @ USER t...
6,1000,xx @ USER @ USER thank you fam ! xx @ USER @ U...
7,1000,rape of texas teen jada @ URL / p0hfdc2wmp @ H...
8,1000,pneumonia in the hospital . he carried me on h...
9,1000,more soon ! merry christmas ! ! ! ! @ USER wil...


In [ ]:
df = pd